In [446]:
import pandas as pd
import numpy as np
import pickle

import nltk
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import json

from warnings import filterwarnings
filterwarnings('ignore')

In [381]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [382]:
pd.set_option('display.max_columns', None)

In [383]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

movies.shape, credits.shape

((4803, 20), (4803, 4))

In [384]:
movies.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466


In [385]:
credits.head(3)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [386]:
data = movies.merge(credits, on='title')
data.shape

(4809, 23)

In [387]:
data.head(3)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,285,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,206647,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [388]:
data.isnull().sum()

budget                     0
genres                     0
homepage                3096
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
movie_id                   0
cast                       0
crew                       0
dtype: int64

In [389]:
data.drop(['homepage', 'tagline'], axis=1, inplace=True)

In [390]:
data.dropna(inplace=True)

### Preprocessing and Feature Engineering textual data

In [391]:
data_t = data[['genres', 'keywords', 'overview', 'production_companies', 'production_countries', 'cast', 'crew']]
data_t.head(3)

,genres,keywords,overview,production_companies,production_countries,cast,crew
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","In the 22nd century, a paraplegic Marine is di...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","Captain Barbossa, long believed to be dead, ha...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",A cryptic message from Bond’s past sends him o...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."


In [392]:
data_t.isnull().sum()

genres                  0
keywords                0
overview                0
production_companies    0
production_countries    0
cast                    0
crew                    0
dtype: int64

In [393]:
data_t['genres'][0]

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [394]:
json.loads('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [395]:
def get_genres(genre_str):
    genres = json.loads(genre_str)
    return [genre['name'] for genre in genres]

In [396]:
data_t['genres'] = data_t['genres'].apply(get_genres)

In [397]:
data_t['keywords'][0]

'[{"id": 1463, "name": "culture clash"}, {"id": 2964, "name": "future"}, {"id": 3386, "name": "space war"}, {"id": 3388, "name": "space colony"}, {"id": 3679, "name": "society"}, {"id": 3801, "name": "space travel"}, {"id": 9685, "name": "futuristic"}, {"id": 9840, "name": "romance"}, {"id": 9882, "name": "space"}, {"id": 9951, "name": "alien"}, {"id": 10148, "name": "tribe"}, {"id": 10158, "name": "alien planet"}, {"id": 10987, "name": "cgi"}, {"id": 11399, "name": "marine"}, {"id": 13065, "name": "soldier"}, {"id": 14643, "name": "battle"}, {"id": 14720, "name": "love affair"}, {"id": 165431, "name": "anti war"}, {"id": 193554, "name": "power relations"}, {"id": 206690, "name": "mind and soul"}, {"id": 209714, "name": "3d"}]'

In [398]:
def get_keywords(keyword_str):
    keywords = json.loads(keyword_str)
    return [keyword['name'] for keyword in keywords]

In [399]:
data_t['keywords'] = data_t['keywords'].apply(get_keywords)

In [400]:
data_t['production_companies'][1]

'[{"name": "Walt Disney Pictures", "id": 2}, {"name": "Jerry Bruckheimer Films", "id": 130}, {"name": "Second Mate Productions", "id": 19936}]'

In [401]:
def get_production(production_str):
    production_list =[]
    production = json.loads(production_str)
    for i in range(len(production)):
        if i == 2:
            break
        else:
            production_list.append(production[i]['name'])
    return production_list

In [402]:
data_t['production_companies'] = data_t['production_companies'].apply(get_production)

In [403]:
data_t['production_countries'][0]

'[{"iso_3166_1": "US", "name": "United States of America"}, {"iso_3166_1": "GB", "name": "United Kingdom"}]'

In [404]:
def get_country(country_str):
    country_list = []
    countries = json.loads(country_str)
    for i in range(len(countries)):
        if i == 2:
            break
        else:
            country_list.append(countries[i]['iso_3166_1'])
    return country_list

In [405]:
data_t['production_countries'] = data_t['production_countries'].apply(get_country)

In [406]:
# data_t['cast'][0]

In [407]:
def get_cast(cast_str):
    cast_list = []
    cast = json.loads(cast_str)
    for i in range(len(cast)):
        if i == 3:
            break
        else:
            cast_list.append(cast[i]['name'])
    return cast_list

In [408]:
data_t['cast'] = data_t['cast'].apply(get_cast)

In [409]:
data_t['crew'][0]

'[{"credit_id": "52fe48009251416c750aca23", "department": "Editing", "gender": 0, "id": 1721, "job": "Editor", "name": "Stephen E. Rivkin"}, {"credit_id": "539c47ecc3a36810e3001f87", "department": "Art", "gender": 2, "id": 496, "job": "Production Design", "name": "Rick Carter"}, {"credit_id": "54491c89c3a3680fb4001cf7", "department": "Sound", "gender": 0, "id": 900, "job": "Sound Designer", "name": "Christopher Boyes"}, {"credit_id": "54491cb70e0a267480001bd0", "department": "Sound", "gender": 0, "id": 900, "job": "Supervising Sound Editor", "name": "Christopher Boyes"}, {"credit_id": "539c4a4cc3a36810c9002101", "department": "Production", "gender": 1, "id": 1262, "job": "Casting", "name": "Mali Finn"}, {"credit_id": "5544ee3b925141499f0008fc", "department": "Sound", "gender": 2, "id": 1729, "job": "Original Music Composer", "name": "James Horner"}, {"credit_id": "52fe48009251416c750ac9c3", "department": "Directing", "gender": 2, "id": 2710, "job": "Director", "name": "James Cameron"},

In [410]:
def get_director(crew_str):
    director_list = []
    crew = json.loads(crew_str)
    for member in crew:
        if member['job'] == 'Director':
            director_list.append(member['name'])
    return director_list

In [411]:
data_t['directors'] = data_t['crew'].apply(get_director)

In [412]:
def get_producers(crew_str):
    producers_list = []
    crew = json.loads(crew_str)

    count =0
    for member in crew:
        if count == 2:
            break
            
        if member['job'] == 'Producer':
            producers_list.append(member['name'])
            count+=1
            
    return producers_list

In [413]:
data_t['producers'] = data_t['crew'].apply(get_producers)

In [414]:
data_t.drop('crew', axis=1, inplace=True)

In [415]:
data_t.head(3)

,genres,keywords,overview,production_companies,production_countries,cast,directors,producers
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","In the 22nd century, a paraplegic Marine is di...","[Ingenious Film Partners, Twentieth Century Fo...","[US, GB]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],"[James Cameron, Jon Landau]"
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","Captain Barbossa, long believed to be dead, ha...","[Walt Disney Pictures, Jerry Bruckheimer Films]",[US],"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],"[Jerry Bruckheimer, Eric McLeod]"
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",A cryptic message from Bond’s past sends him o...,"[Columbia Pictures, Danjaq]","[GB, US]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],"[Barbara Broccoli, Michael G. Wilson]"


In [416]:
data_t['production_companies'] = data_t['production_companies'].apply(lambda x: [i.replace(' ', '') for i in x])
data_t['cast'] = data_t['cast'].apply(lambda x: [i.replace(' ', '') for i in x])
data_t['directors'] = data_t['directors'].apply(lambda x: [i.replace(' ', '') for i in x])
data_t['producers'] = data_t['producers'].apply(lambda x: [i.replace(' ', '') for i in x])

In [417]:
data_t['overview'] = data_t['overview'].astype(str)
data_t['overview'] = data_t['overview'].apply(lambda x: x.split())

In [418]:
data_t['textual_data'] = data_t['genres']+data_t['keywords']+data_t['overview']+data_t['production_companies']+data_t['cast']+data_t['directors']+data_t['producers']

In [419]:
data_t.head(3)

,genres,keywords,overview,production_companies,production_countries,cast,directors,producers,textual_data
0,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[In, the, 22nd, century,, a, paraplegic, Marin...","[IngeniousFilmPartners, TwentiethCenturyFoxFil...","[US, GB]","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron],"[JamesCameron, JonLandau]","[Action, Adventure, Fantasy, Science Fiction, ..."
1,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Captain, Barbossa,, long, believed, to, be, d...","[WaltDisneyPictures, JerryBruckheimerFilms]",[US],"[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski],"[JerryBruckheimer, EricMcLeod]","[Adventure, Fantasy, Action, ocean, drug abuse..."
2,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[A, cryptic, message, from, Bond’s, past, send...","[ColumbiaPictures, Danjaq]","[GB, US]","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes],"[BarbaraBroccoli, MichaelG.Wilson]","[Action, Adventure, Crime, spy, based on novel..."


In [420]:
data_new = pd.concat([data[['id', 'title']], data_t['textual_data']], axis=1)
data_new

,id,title,textual_data
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction, ..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action, ocean, drug abuse..."
2,206647,Spectre,"[Action, Adventure, Crime, spy, based on novel..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller, dc comics, cr..."
4,49529,John Carter,"[Action, Adventure, Science Fiction, based on ..."
...,...,...,...
4804,9367,El Mariachi,"[Action, Crime, Thriller, united states–mexico..."
4805,72766,Newlyweds,"[Comedy, Romance, A, newlywed, couple's, honey..."
4806,231617,"Signed, Sealed, Delivered","[Comedy, Drama, Romance, TV Movie, date, love ..."
4807,126186,Shanghai Calling,"[When, ambitious, New, York, attorney, Sam, is..."


In [421]:
data_new['textual_data'].dtype

dtype('O')

In [422]:
data_new['textual_data'] = data_new['textual_data'].apply(lambda x: ' '.join(x)).apply(lambda x: x.lower())
data_new.head(3)

,id,title,textual_data
0,19995,Avatar,action adventure fantasy science fiction cultu...
1,285,Pirates of the Caribbean: At World's End,adventure fantasy action ocean drug abuse exot...
2,206647,Spectre,action adventure crime spy based on novel secr...


In [423]:
data_new.isnull().sum()

id              0
title           0
textual_data    0
dtype: int64

### Stemming textual data

In [424]:
stemmer = PorterStemmer()

In [425]:
def stem_text(text):
    words = nltk.word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return ' '.join(stemmed_words)

In [426]:
data_new['textual_data'] = data_new['textual_data'].apply(stem_text)

In [427]:
data_new['textual_data'][0]

'action adventur fantasi scienc fiction cultur clash futur space war space coloni societi space travel futurist romanc space alien tribe alien planet cgi marin soldier battl love affair anti war power relat mind and soul 3d in the 22nd centuri , a parapleg marin is dispatch to the moon pandora on a uniqu mission , but becom torn between follow order and protect an alien civil . ingeniousfilmpartn twentiethcenturyfoxfilmcorpor samworthington zoesaldana sigourneyweav jamescameron jamescameron jonlandau'

### Vectorization

In [428]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf.fit_transform(data_new['textual_data'])

In [429]:
tfidf_matrix.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Cosine Similarity

In [430]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [431]:
cosine_sim

array([[1.        , 0.02056034, 0.01918893, ..., 0.04088475, 0.00434217,
        0.        ],
       [0.02056034, 1.        , 0.01003306, ..., 0.02589156, 0.        ,
        0.0055775 ],
       [0.01918893, 0.01003306, 1.        , ..., 0.01885924, 0.        ,
        0.        ],
       ...,
       [0.04088475, 0.02589156, 0.01885924, ..., 1.        , 0.02397053,
        0.02625699],
       [0.00434217, 0.        , 0.        , ..., 0.02397053, 1.        ,
        0.02007555],
       [0.        , 0.0055775 , 0.        , ..., 0.02625699, 0.02007555,
        1.        ]])

### Recommender

In [432]:
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = data_new[data_new['title'] == title].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return data_new['title'].iloc[movie_indices]

In [433]:
print(get_recommendations('Avatar'))

2409             Aliens
4338     Silent Running
838              Alien³
3163              Alien
373     Mission to Mars
Name: title, dtype: object


In [436]:
print(get_recommendations('The Dark Knight Rises'))

65      The Dark Knight
119       Batman Begins
1363             Batman
428      Batman Returns
1362             Batman
Name: title, dtype: object


In [438]:
print(get_recommendations('Spectre'))

11          Quantum of Solace
29                    Skyfall
147           Die Another Day
1346    Never Say Never Again
3290                 Restless
Name: title, dtype: object


In [440]:
print(get_recommendations('Guardians of the Galaxy'))

126          Thor: The Dark World
7         Avengers: Age of Ultron
182                       Ant-Man
129                          Thor
26     Captain America: Civil War
Name: title, dtype: object


In [443]:
print(get_recommendations('Captain America: Civil War'))

7                  Avengers: Age of Ultron
31                              Iron Man 3
85     Captain America: The Winter Soldier
16                            The Avengers
169     Captain America: The First Avenger
Name: title, dtype: object


In [451]:
with open('cosine_sim.pkl', 'wb') as f:
    pickle.dump(cosine_sim, f)